# Compare alignment of the different versions of the reference atlases

## Description

The mouse brain reference volumes is a pair volumetric files from the Allen Institute for Brain Science (AIBS): the **Nissl** and the **annotations** volumes. 

The **Nissl volume** is used to obtain the cell distribution in the mouse brain.   
The **annotations volume** provides a brain region id to each voxel of the Nissl volume and any volumetric dataset of the AIBS. The mouse brain regions are organized in a hierarchical tree, with leaves representing the finest subdivisions captured by the AIBS. This **hierarchy** is stored in a **json file** and is common to each reference atlas version. It is also available on the AIBS website.

These two whole brain volumes exist in multiple versions and need to be aligned to each other to produce the best results in terms of cell density estimation (see https://doi.org/10.1101/2021.11.20.469384 for more details).

To test the alignment between the different pairs of annotations volume and Nissl volume, we are plotting the annotations volumes in color on top of the Nissl volumes. 

In [ ]:
import numpy as np
from os.path import join
from voxcell import RegionMap, VoxelData
import matplotlib.pylab as plt
import matplotlib.patches as patches
from scipy.ndimage import binary_dilation, generate_binary_structure

Set below the path to the folder containing the data, including the annotations volumes, the Nissl volumes and the json hierarchy file.

Set also the folder in which the figures produced will be stored.

We are comparing here three versions of the reference atlas pairs that we store in subfolders.

In [ ]:
DATA_FOLDER = 'path/to/data/'
OUTPUT_FOLDER = "/path/to/output/"
ccfs = ["bbp", "ccfv2", "ccfv3"]  # subfolders containing the annotations and Nissl volumes
ccf_fullname = ['CCFbbp', 'CCFv2', 'CCFv3']  # names to display on the figure
annotation_file = "annotations.nrrd"  # name given to each nrrd annotation file
nissl_file = "ara_nissl_25.nrrd"  # name given to each nrrd Nissl file
region_hierarchy_file = "brain_regions.json"  # mouse brain region hierarchy file (json).

## Set matplotlib parameters

In [ ]:
SMALL_SIZE = 48
MEDIUM_SIZE = 48
BIGGER_SIZE = 48
plt.rcParams["font.family"] = "Arial"
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

plt.rcParams['xtick.major.pad']  = 9.0
plt.rcParams['axes.linewidth'] = 2.0
plt.rcParams['xtick.major.size' ] = 7.0*3.0
plt.rcParams['xtick.minor.size' ] = 4.0*3.0
plt.rcParams['ytick.major.size' ] = 7.0*3.0
plt.rcParams['ytick.minor.size' ] = 4.0*3.0
plt.rcParams['xtick.major.width'] = 2.4
plt.rcParams['xtick.minor.width'] = 1.8
plt.rcParams['ytick.major.width'] = 2.4
plt.rcParams['ytick.minor.width'] = 1.8

# color dictionary
CBcdict={
    'Bl':np.array([0,0,0]),
    'Or':np.array([.9,.6,0]),
    'SB':np.array([.35,.7,.9]),
    'bG':np.array([0,.6,.5]),
    'Ye':np.array([.95,.9,.25]),
    'Bu':np.array([0,.45,.7]),
    'Ve':np.array([.8,.4,0]),
    'rP':np.array([.8,.6,.7]),
}

## Plot coronal slice of annotations on top of Nissl 

In [ ]:
cmap = plt.cm.gray  # cmap for Nissl
struct = generate_binary_structure(3, 3)
thickness = 1 # thickness of the boundary between regions
region_map = RegionMap.load_json(join(DATA_FOLDER, region_hierarchy_file)) # load json hierarchy file

### Plot cerebellar cortex, granular and molecular layers

In [ ]:
ids_cereb = region_map.find("Cerebellar cortex", attr="name", with_descendants=True)
ids_layers = np.vstack((list(ids_cereb & region_map.find("@.*molecular layer$", attr="name", with_descendants=True)),
                             list(ids_cereb & region_map.find("@.*granular layer$", attr="name", with_descendants=True))))
ids_parents = np.array([region_map.get(id_, attr="parent_structure_id") for id_ in ids_layers[0]])
colors = {}
for i, (id_parent, (id_mol, id_gran)) in enumerate(zip(ids_parents, ids_layers.T)):
    color = list(CBcdict.values())[(i +1) % 7 + 1]
    colors[id_parent] = color 
    colors[id_mol] = color * 0.5 
    colors[id_gran] = color * 0.8

In [ ]:
slice_ = 475  # coronal slice id showing cerebellum
fig = plt.Figure(figsize=(30, 12))

for i, ccf in enumerate(ccfs):
    annotation = VoxelData.load_nrrd(join(DATA_FOLDER, ccf, annotation_file)).raw
    nissl = VoxelData.load_nrrd(join(DATA_FOLDER, ccf, nissl_file)).raw
    loc_slice = slice_

    img = nissl[loc_slice, 42:annotation.shape[1] // 2+50, 50:annotation.shape[2] // 2]
    ann_img = annotation[loc_slice, 42:annotation.shape[1] // 2+50, 50:annotation.shape[2] // 2]

    norm = plt.Normalize(nissl.min(), nissl.max())
    rgba = cmap(1-norm(img))
    annotation[loc_slice-1] = 0 # Force annotation for 3d boundary
    annotation[loc_slice+1] = 0

    ann_filt = np.isin(annotation, ids_layers[0])
    boundaries = binary_dilation(ann_filt, struct, iterations=thickness) * (np.isin(annotation, ids_layers[1]))
    boundaries = boundaries[loc_slice, 42:annotation.shape[1] // 2+50, 50:annotation.shape[2] // 2] 
    rgba += 0.4
    for id_, color in colors.items():
        rgba[ann_img==id_, :3] -= color * 0.7
    rgba[boundaries, :3] = CBcdict["Ye"] + 0.3

    rgba = np.maximum(rgba/np.max(rgba), 0)
    ax = plt.subplot2grid((1,3), (0, i), rowspan=1, colspan=1)
    ax.imshow(rgba, interpolation='nearest')
    if i == 0:
        x = 12
        y = 15
        ax.hlines(y=y, xmin=x, xmax=x+40, linewidth=15, color='black')
        ax.text(x+18,y-5, "1 mm", weight='bold', horizontalalignment='center')

    ax.axis('off')
    ax.set_title(ccf_fullname[i])

plt.tight_layout()
plt.savefig(join(OUTPUT_FOLDER, 'annotation_Nissl_Cereb.png'), dpi=200, facecolor="white")

### Plot Isocortex layers 1 and 2

In [ ]:
ids_ctx = region_map.find("Isocortex", attr="name", with_descendants=True)
ids_l1 = ids_ctx & region_map.find("@.*1[ab]?$", attr="acronym", with_descendants=True)
ids_l2 = ids_ctx & region_map.find("@.*2.*[ab]?$", attr="acronym", with_descendants=True)
colors = np.array([1-CBcdict["Bu"], 1-CBcdict["Or"]]) 

In [ ]:
slice_ = 195  # coronal slice id showing L1 L2 in the cortex
fig = plt.Figure(figsize=(30, 12))

for i, ccf in enumerate(ccfs):
    annotation = VoxelData.load_nrrd(join(DATA_FOLDER, ccf, annotation_file)).raw
    nissl = VoxelData.load_nrrd(join(DATA_FOLDER, ccf, nissl_file)).raw
    loc_slice = slice_
    
    img = nissl[loc_slice, 30:annotation.shape[1] // 2-50, 50:annotation.shape[2] // 2]
    ann_img = np.copy(annotation[loc_slice, 30:annotation.shape[1] // 2-50, 50:annotation.shape[2] // 2])

    norm = plt.Normalize(nissl.min(), nissl.max())
    rgba = cmap(1-norm(img))
    annotation[loc_slice-1] = 0 # Force annotation for 3d boundary
    annotation[loc_slice+1] = 0
    
    filter_l1 = np.isin(ann_img, list(ids_l1))
    filter_l2 = np.isin(ann_img, list(ids_l2))
    ann_img *= 0
    ann_img[filter_l1] = 1
    ann_img[filter_l2] = 2
    ann_img[:, :60] = 0
    ann_filt = np.isin(annotation, list(ids_l1))
    boundaries = binary_dilation(ann_filt, struct, iterations=thickness)
    boundaries = boundaries[loc_slice, 30:annotation.shape[1] // 2-50, 50:annotation.shape[2] // 2]
    boundaries *= filter_l2
    boundaries[:, :60] = False
    rgba-=0.5
    rgba[ann_img>0, :3] -= 0.3 * colors[ann_img[ann_img>0]-1]    
    rgba[boundaries, :3] = CBcdict["Ye"] - 0.5

    rgba = np.maximum(rgba/np.max(rgba), 0)
    ax = plt.subplot2grid((1,3), (0, i), rowspan=1, colspan=1)
    ax.imshow(rgba, interpolation='nearest')
    ax.vlines(x=60, ymin=0, ymax=rgba.shape[0]-1, linewidth=7, color="black")
    if i == 0:
        x = 12
        y = 15
        ax.hlines(y=y, xmin=x, xmax=x+40, linewidth=15, color="black")
        ax.text(x+18,y-5, "1 mm", weight='bold', horizontalalignment='center')
    elif i>=1:
        rect = patches.Rectangle((60, 25), 10, 10, fill=False, linewidth=6)
        ax.add_patch(rect)
        ax.plot([60, 73], [35, 79], linewidth=6, color='black')
        ax.plot([70, 108], [25, 42], linewidth=6, color='black')
                
        a = plt.axes([0.8 - 2/3 +i/3 + (0.01 if i==1 else 0), .335, .06, .15])
        a.axis("off")
        a.imshow(rgba[25:35,60:70], interpolation='nearest')
        autoAxis = a.axis()
        rec = plt.Rectangle((autoAxis[0],autoAxis[2]),(autoAxis[1]-autoAxis[0]),(autoAxis[3]-autoAxis[2]),fill=False,lw=6)
        rec = a.add_patch(rec)
        rec.set_clip_on(False)
        
        rect = patches.Rectangle((166, 30), 10, 10, fill=False, linewidth=6)
        ax.add_patch(rect)
        ax.plot([166, 130], [30, 42], linewidth=6, color='black')
        ax.plot([176, 165], [40, 79], linewidth=6, color='black')
        a = plt.axes([0.895 - 2/3 + i/3 + (0.01 if i==1 else 0), .335, .06, .15])
        a.axis("off")
        a.imshow(rgba[30:40, 166:176], interpolation='nearest')
        autoAxis = a.axis()
        rec = plt.Rectangle((autoAxis[0],autoAxis[2]),(autoAxis[1]-autoAxis[0]),(autoAxis[3]-autoAxis[2]),fill=False,lw=6)
        rec = a.add_patch(rec)
        rec.set_clip_on(False)

    ax.axis('off')
    ax.set_title(ccf_fullname[i])

plt.tight_layout()
plt.savefig(join(OUTPUT_FOLDER, 'annotation_Nissl_L12.png'), dpi=200, facecolor="white")